In [1]:
# just update driver and run, will create folder with date in "downloads" folder
### development
# create database to store the data


from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import os
import wget
import sqlite3
from bs4 import BeautifulSoup as bs
from lxml import etree
chrome_options = Options()
chrome_options.add_argument("--headless")
headers = {
    #'User-Agent':str(ua.chrome)
    'User-Agent':'Mozilla/5.0 AppleWebKit/537.36 (KHTML, like Gecko; Google Web Preview Analytics) Chrome/41.0.2272.118 Safari/537.36 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)'
}

###Date
from datetime import datetime
from datetime import timedelta
dt = datetime.today()
year = str(dt.year)
month = str(dt.month).zfill(2)
day = str(dt.day).zfill(2)
date = year+ '-' + month +'-' + day

###Input
folder = 'downloads/' + date +'/'
try:
    os.mkdir(folder)
except:
    None

def download_iv_file(stock_code):
    
    url = 'https://www.hkex.com.hk/chi/sorc/options/statistics_hv_iv_c.aspx?action=csv&type=3&ucode=' + stock_code
    destination = folder + stock_code + '_' + date + '.csv'
    wget.download(url,out=destination)
    
    
        
def underlying_list():
    date_list = []
    date_value = []
    if os.name == 'nt':
        driver = webdriver.Chrome('./chromedriver (2).exe',chrome_options=chrome_options)
    else:
        driver = webdriver.Chrome('./chromedriver',chrome_options=chrome_options)
    driver.set_window_size(1440, 900)
    
    url = 'https://www.hkex.com.hk/chi/sorc/options/statistics_hv_iv_c.aspx'
    driver.get(url)
    soup = bs(driver.page_source,'lxml')
    lastpage = soup.find("table",{"class":"noncms_table"})
    lastpage2 = lastpage.find("select")
    #print(lastpage2)
    date_option = lastpage2.find_all('option')
    driver.close()
    for x in date_option:
        date_list.append(x['value'])
        date_value.append(x.text)
    return date_list

for x in underlying_list():
    download_iv_file(x)



<ipython-input-1-070f7c32a0e8>:53: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome('./chromedriver',chrome_options=chrome_options)


In [43]:
# download the files and only import the latest record into sql database
import pandas as pd
import numpy as np


result = []

conn = sqlite3.connect('data.db')



folder_name = os.listdir('downloads')
for x in folder_name:
    try:
        sql_data = pd.read_sql('select * from data', conn) 

        # check the latest date of the data
        latest_date = sql_data['交易日'].max()
    except:
        latest_date = np.array(np.datetime64('1900-01-01')) 
    
    if not x.startswith('.'):
        files = os.listdir('downloads/' + x)
    else:
        continue
    for y in files:
        if not y.startswith('.'):
            df = pd.read_csv('downloads/'+x+'/'+y, skiprows=1)
            df['ticker'] = y[0:5]
            df['交易日'] = df['交易日'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y'))
            df = df[df['交易日']>latest_date]

            result.append(df)
            df.to_sql('data',conn,if_exists='append')

# the order is not from oldest folder first!!!!!

/Users/kit/.local/share/virtualenvs/monitor-t9hVwawW/lib/python3.8/site-packages/pandas/core/generic.py:2778: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [44]:
# READ file
#folder_name
data = pd.read_sql('select * from data',conn)

In [47]:
#data[data['ticker']=='03690']
folder_name

['.DS_Store', '2021-02-01', '2021-01-24']